# Classic Design of Experiments I

In this notebook, we will learn about the basics of the classic design of experiments.

We start with an empirical analysis of **Hotelling's experiment** and end with a comparison of the **OFAT** and an **own DoE method**.

### **Table of Contents**
1. [Hottelling's Experiment](#hotelling-experiment)
2. [One-factor-at-a-time Method](#one-factor-at-time-method)
3. [Own Idea of a DoE Method](#own-idea-of-a-doe-method)

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from e2ml.experimentation import (
    get_hotellings_experiment_measurements,
    get_hotellings_experiment_errors,
    black_box_data_generation,
)

### **1. Hotelling's Experiment** <a class="anchor" id="hotelling-experiment"></a>
This section empirically studies Hotelling's experiment extended to 16 objects. In the first step, we create the corresponding design/factor matrices for the OFAT and combinatorial method.

In [ ]:
# Create Hadamard `X_hadamard` and identity matrix `X_identity`.
# BEGIN SOLUTION
from scipy.linalg import hadamard
X_hadamard = hadamard(16)
X_identity = np.eye(16)
# END SOLUTION

Next, we implement a function to estimate the object weights from the experimental results obtained after executing the experimental trials according to a given design matrix.

In [ ]:
def estimate_object_weights(X, y):
    """
    Gets a design matrix of object configurations and corresponding measurements as inputs
    to estimate the object weights.

    Parameters
    ----------
    X : numpy.ndarray of shape (16, 16)
        Design matrix of object configurations.
    y : numpy.ndarray of shape (16,)
        Measurements.

    Returns
    -------
    theta_hat : numpy.ndarray of shape (16,)
        Estimate object weights.
    """
    # BEGIN SOLUTION
    theta_hat = np.linalg.inv(X) @ y
    return theta_hat
    # END SOLUTION

As the final step, we replicate the experiments for the OFAT and combinatorial method to estimate the expected sum of squared differences between true object and estimated object weights according to:

BEGIN SOLUTION

$$
E\left(||\boldsymbol{\theta} - \boldsymbol{\hat{\theta}}||_2^2\right) \approx \frac{1}{K} \sum_{k=1}^{K} ||\boldsymbol{\theta} - \boldsymbol{\hat{\theta}}_k||_2^2,
$$
where $\boldsymbol{\hat{\theta}}_k, k \in \{1, \dots, K\}$ denotes the estimated object weights in the $k$-th replication of the experiment.

END SOLUTION

In [ ]:
# Replicate the experiment 10,000 times for both design matrices
# to visualize the error distribution of the estimated weights via a histogram
# and compute the empirical mean of the sum of squared differences.
# BEGIN SOLUTION
n_reps = int(1e4)
errors_hadamard = np.zeros(n_reps)
errors_identity = np.zeros(n_reps)
thetas = []
for rep in range(n_reps):
    y_hadamard = get_hotellings_experiment_measurements(X_hadamard)
    y_identity = get_hotellings_experiment_measurements(X_identity)
    theta_hat_hadamard = estimate_object_weights(X_hadamard, y_hadamard)
    theta_hat_identity = estimate_object_weights(X_identity, y_identity)
    errors_hadamard[rep] = get_hotellings_experiment_errors(theta_hat_hadamard)
    errors_identity[rep] = get_hotellings_experiment_errors(theta_hat_identity)
    thetas.append(theta_hat_identity)
plt.figure(figsize=(8, 6))
plt.title(f"Hadamard: {errors_hadamard.mean().round(2)}, Identity: {errors_identity.mean().round(2)}")
plt.hist(errors_hadamard, alpha=0.5, bins=100, label="Hadamard")
plt.hist(errors_identity, alpha=0.5, bins=100, label="Identity")
plt.legend()
plt.show()
# END SOLUTION

#### **Question:**
1. (a) What are the expected sums of squared differences between estimated and true object weights for the OFAT and combinatorial method in the case of 16 objects?

   BEGIN SOLUTION

   For the OFAT method, the expected sum of squared differences increases for 16 objects to:

   $$
   E\left(||\boldsymbol{\theta} - \boldsymbol{\hat{\theta}}||_2^2\right) = \sum_{d=1}^{16} V(\hat{\theta}_d) = \sum_{d=1}^{16} 1 = 16.
   $$

   In contrast, for the combinatorial method with the Hadamard matrix the expected sum of squared difference stays unchanged according to:

   $$
   E\left(||\boldsymbol{\theta} - \boldsymbol{\hat{\theta}}||_2^2\right) = \sum_{d=1}^{16} V(\hat{\theta}_d) = \sum_{d=1}^{16} \frac{1}{16} = 1.
   $$

   END SOLUTION
   
### **2. One-factor-at-a-time Method** <a class="anchor" id="one-factor-at-time-method"></a>
Now, we implement the standard OFAT method for multiple factors and levels. Therefore, we consider the factors $x_1, \dots, x_D$, where each factor has a finite number of levels, i.e., $x_d \in \{0, \dots, L_d-1\}, L_d \in \mathbb{N}_{>0}, d \in \{1, \dots, D\}$, where $x_1 = \dots = x_D = 0$ is defined as standard condition without loss of generality.

2. (a) How many conditions, i.e., the number of rows in the design matrix, will be generated by a standard OFAT method at maximum?

   BEGIN SOLUTION

   There are $L_d$ levels for each factor $x_d$. Iterating over each factor independently according to respective number of levels would correspond to the sum of all number of levels. However, this sum counts the standard condition $D$ instead of one time. Accordingly, we need to subtract $D-1$ from the sum, which results in
   
   $$\sum_{d=1}^{D} (L_d) - (D-1)$$

   as maximum number of conditions.
   
   END SOLUTION
   
We need to implement the corresponding function [`one_factor_at_a_time`](../e2ml/experimentation/_one_factor_at_a_time.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage.
Once, the implementation has been completed, we check our implementation's validity. 

In [ ]:
from e2ml.experimentation import one_factor_at_a_time

levels = [2, 3, 4]

# Obtain the design matrix `X` for the given levels.
X = one_factor_at_a_time(levels=levels) # <-- SOLUTION

assert X.shape[0] == 7, "The number of conditions is wrong."
assert X.shape[1] == 3, "The number of factors is wrong."
assert ~(((X > 0).sum(axis=-1) > 1).any()), "There is one row with more than one change factor."
for l_idx, l in enumerate(levels):
    assert np.array_equal(np.unique(X[:, l_idx]), np.arange(l)), f"No all levels were tested of factor {l_idx}."

Once we have implemented and tested our OFAT function, we apply it to a black-box experiment via the function `black_box_data_generation`. In total, there are
$D=4$ factors with $L_1 = 20, L_2 = 4, L_3 = 20, L_4 = 5$. For these factors, we generate a design matrix $\mathbf{X}$ according to a full OFAT method and obtain the corresponding measurements. The goal is to find a condition maximizing the response of the experiment.

In [ ]:
# Generate levels according to the black-box experiment.
levels = [20, 4, 20, 5] # <-- SOLUTION

# Obtain the design matrix `X` for the given levels.
X = one_factor_at_a_time(levels=levels) # <-- SOLUTION

# Perform experimental trials for the given matrix `X` and save the results `res`.
res = black_box_data_generation(X) # <-- SOLUTION

# Plot the obtained results as a histogram.
# BEGIN SOLUTION
plt.figure(figsize=(8, 6))
plt.title(f"OFAT Black-box Experiment Results")
plt.hist(res, alpha=0.5)
plt.show()
# END SOLUTION

# Print the best condition and its maximum score.
# BEGIN SOLUTION
max_idx = res.argmax()
print(f"Best condition: {X[max_idx]}")
print(f"Best score: {res[max_idx]}")
# END SOLUTION

### **3. Own Idea of a DoE Method** <a class="anchor" id="own-idea-of-a-doe-method"></a>
A substantial disadvantage of the OFAT method is that it ignores any interactions between varying levels of factors. Therefore, we aim to implement an own DoE method `own_doe_method`, which advances the OFAT method. The DOE method should allow restricting the number of conditions to a user-defined value. Implement and compare your method with the results of OFAT, when using the same number of experimental trials.

In [ ]:
# Implement `own_doe_method` taking `levels` as input.
# BEGIN SOLUTION
from sklearn.utils import column_or_1d, check_scalar, check_random_state
def own_doe_method(levels, n_conditions, random_state=42):
    """
    Implements a random generation of conditions for given factor-wise levels.

    Parameters
    ----------
    levels : array-like of shape (n_factors,)
        Integer array indicating the number of levels of each input design factor (variable).
    n_conditions : int
        Number of conditions to be generated.
    random_state : int
        Ensures reproducibility.

    Returns
    -------
    X : np.ndarray of shape (n_combs, n_factors)
        Design matrix with coded levels 0 to k-1 for a k-level factor.
    """
    levels = column_or_1d(levels, dtype=int)
    check_scalar(levels.min(), min_val=1, name="minimum level per factor", target_type=np.int64)
    X = []
    random_state = check_random_state(random_state)
    for i in range(n_conditions):
        is_old = True
        while is_old:
            candidate = [random_state.choice(np.arange(l)) for l in levels]
            if is_old not in X:
                is_old = False
        X.append(candidate)
    return np.array(X).astype(int)
# END SOLUTION

# Obtain the design matrix `X` for the given levels.
X = own_doe_method(levels=levels, n_conditions=46) # <-- SOLUTION

# Perform experimental trials for the given matrix `X` and save the results `res`.
res = black_box_data_generation(X) # <-- SOLUTION

# Plot the obtained results as a histogram.
# BEGIN SOLUTION
plt.figure(figsize=(8, 6))
plt.title(f"Own DOE Method Black-box Experiment Results")
plt.hist(res, alpha=0.5)
plt.show()
# END SOLUTION

# Print the best condition and its maximum score.
# BEGIN SOLUTION
max_idx = res.argmax()
print(f"Best condition: {X[max_idx]}")
print(f"Best score: {res[max_idx]}")
# END SOLUTION